# Dimensionality Reduction of VGG16
In this tutorial we will present how to create a reduced version of VGG16 using the techniques described in the article ''A Dimensionality Reduction Approach for Convolutional Neural Networks'', Meneghetti L., Demo N., Rozza G., https://arxiv.org/abs/2110.09163 (2021)

In [1]:
import torch
import numpy as np
import torchvision

import torchvision.transforms as transforms
import torchvision.datasets as datasets
import pandas as pd

## Loading of the model
First of all we need to load the model we want to use (in this case VGG16) starting from a checkpoint file, i.e. a file containing the status of the model after a training process with a chosen dataset. Here we will use the CIFAR10 dataset, but we will also show how to generalize everythong using a custom dataset.

It is important to highlight that the models of VGG-nets implemented in PyTorch (https://pytorch.org/hub/pytorch_vision_vgg/), e.g. 
````
model = torch.hub.load('pytorch/vision:v0.10.0', 'vgg16', pretrained=True),
```
are models pre-trained on the ImageNet dataset, that consists of images of dimensions 224x224. Therefore, in order to use datasets like the CIFAR10, composed of images 32x32, we need to change the architecture of VGG-nets, as was done in the file 'vgg.py'. 

In [2]:
import sys
sys.path.insert(0, '../')
from smithers.ml.vgg import VGG
from smithers.ml.utils import get_seq_model

if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')
    

pretrained = '../checkpoint_vgg16_custom20.pth.tar'
#model = VGG(None, classifier='cifar', num_classes=4, init_weights=False, pretrain_weights=pretrained)
model = torch.load(pretrained)
model = model['model']
seq_model = get_seq_model(model)

## Loading of the dataset
### CIFAR10 Dataset
As stated before, we use the CIFAR10 dataset (already implemented in PyTorch) to test our technique. It is a computer-vision dataset used for object recognition. It consists of 60000 32 × 32 colour images divided in 10 non-overlapping classes: airplane, automobile, bird, cat, deer, dog, frog, horse, ship, and truck.

See https://www.cs.toronto.edu/~kriz/cifar.html for more details on this dataset and on how to download it.

In [ ]:
#load CIFAR10 dataset for training and testingpretrained = '../checkpoint_vgg16_custom20.pth.tar'
batch_size = 8 #this can be changed
data_path = '../datasets/' 
# transform functions: take in input a PIL image and apply this
# transformations
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])
train_dataset = datasets.CIFAR10(root=data_path + 'CIFAR10/',
                                 train=True,
                                 download=True,
                                 transform=transform_train)
train_loader = torch.utils.data.DataLoader(train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True)
test_dataset = datasets.CIFAR10(root=data_path + 'CIFAR10/',
                                train=False,
                                download=True,
                                transform=transform_test)
test_loader = torch.utils.data.DataLoader(test_dataset,
                                          batch_size=batch_size,
                                          shuffle=True)
train_labels = torch.tensor(train_loader.dataset.targets)
targets = list(train_labels)

### Custom dataset
If we want to use a custom dataset, we need firstly to construct it, following for example the tutorial on the construction of a custom dataset for the problem of Image Recognition. Hence, the previuous cell will be substitute with the following one.

In [3]:
from torch.utils.data.sampler import SubsetRandomSampler
from collections import OrderedDict
from smithers.ml.imagerec_dataset import Imagerec_Dataset

# load custom dataset for training and testing
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

data = pd.read_csv('../dataset_imagerec/dataframe.csv')
data_path = '../dataset_imagerec/'
# SPLIT OF THE DATASET
batch_size = 128
validation_split = .2
shuffle_dataset = True
random_seed = 42

dataset_size = len(data)
indices = list(range(dataset_size))
split = int(np.floor(validation_split * dataset_size))
if shuffle_dataset:
    np.random.seed(random_seed)
    np.random.shuffle(indices)
train_indices, val_indices = indices[split:], indices[:split]
print('train data', len(train_indices))
train_sampler = SubsetRandomSampler(train_indices)
valid_sampler = SubsetRandomSampler(val_indices)
resize_dim = [32, 32]

dataset_imagerec = Imagerec_Dataset(data, data_path, resize_dim, transform)
train_dataset = dataset_imagerec.getdata(train_indices)
train_loader = torch.utils.data.DataLoader(dataset_imagerec,
                                           batch_size=batch_size,
                                           sampler=train_sampler)
test_loader = torch.utils.data.DataLoader(dataset_imagerec,
                                          batch_size=batch_size,
                                          sampler=valid_sampler)

classes = ('Cups', 'Dishes', 'Glass', 'Mixed')
#classes = ('class_1', 'class_2', 'class_3', 'class_4')
n_class = len(classes)
targets = list(dataset_imagerec.targets[train_indices])
train_labels = torch.tensor(targets)

train data 2759


/scratch/lmeneghe/anaconda/envs/prova/lib/python3.8/site-packages/torchvision/transforms/functional.py:135: UserWarning: Legacy tensor constructor is deprecated. Use: torch.tensor(...) for creating tensors from tensor-like objects; or torch.empty(...) for creating an uninitialized tensor with specific sizes. (Triggered internally at  /opt/conda/conda-bld/pytorch_1617865552725/work/torch/csrc/utils/tensor_new.cpp:477.)
  img = torch.ByteTensor(torch.ByteStorage.from_buffer(pic.tobytes()))


## Reduction of VGG16
We now perform the reduction of VGG16 using the module NetAdapter. In this case we use 5 as cut off index and 50 as dimension of the reduced space. For the reduced method and the input-output mapping there are two different choices: 'POD' and 'AS' for the first one, and 'PCE' or 'FNN' for the latter.

In [4]:
total = 0
correct = 0
count = 0
seq_model.eval()
for test, y_test in iter(test_loader):
#Calculate the class probabilities (softmax) for img
    with torch.no_grad():
        output = seq_model(test)
        ps = torch.exp(output)
        _, predicted = torch.max(output.data,1)
        total += y_test.size(0)
        correct += (predicted == y_test).sum().item()
        count += 1
        print("Accuracy of network on test images is {:.4f}....count: {}".format(100*correct/total,  count ))


Accuracy of network on test images is 98.4375....count: 1
Accuracy of network on test images is 97.6562....count: 2
Accuracy of network on test images is 98.1771....count: 3
Accuracy of network on test images is 97.8516....count: 4
Accuracy of network on test images is 97.9688....count: 5
Accuracy of network on test images is 98.1132....count: 6


In [5]:
from smithers.ml.netadapter import NetAdapter
cutoff_idx = 7 
red_dim = 50 
red_method = 'POD' 
inout_method = 'FNN'
n_class = 4
netadapter = NetAdapter(cutoff_idx, red_dim, red_method, inout_method)
red_model = netadapter.reduce_net(seq_model, train_dataset, train_labels, train_loader, n_class)
print(red_model)

RedNet(
  (premodel): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=True)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=True)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=

In [7]:
import os
from smithers.ml.utils import Total_param, Total_flops
from smithers.ml.utils import compute_loss, train_kd

rednet_storage = torch.zeros(3)
rednet_flops = torch.zeros(3)

rednet_storage[0], rednet_storage[1], rednet_storage[2] = [
    Total_param(red_model.premodel),
    Total_param(red_model.proj_model),
    Total_param(red_model.inout_map)]

rednet_flops[0], rednet_flops[1], rednet_flops[2] = [
    Total_flops(red_model.premodel, device),
    Total_flops(red_model.proj_model, device),
    Total_flops(red_model.inout_map, device)]

total = 0
correct = 0
count = 0

for test, y_test in iter(test_loader):
#Calculate the class probabilities (softmax) for img
    with torch.no_grad():
        output = red_model(test)
        ps = torch.exp(output)
        _, predicted = torch.max(output.data,1)
        total += y_test.size(0)
        correct += (predicted == y_test).sum().item()
        count += 1
        print("Accuracy of network on test images is {:.4f}....count: {}".format(100*correct/total,  count ))

print(
      'Pre nnz = {:.2f}, proj_model nnz={:.2f}, FNN nnz={:.4f}'.format(
      rednet_storage[0], rednet_storage[1],
      rednet_storage[2]))
print(
      'flops:  Pre = {:.2f}, proj_model = {:.2f}, FNN ={:.2f}'.format(
       rednet_flops[0], rednet_flops[1], rednet_flops[2]))

optimizer = torch.optim.Adam([{
            'params': red_model.premodel.parameters(),
            'lr': 1e-4
            }, {
            'params': red_model.proj_model.parameters(),
            'lr': 1e-5
            }, {
            'params': red_model.inout_map.parameters(),
            'lr': 1e-5
            }])

train_loss = []
test_loss = []
train_loss.append(compute_loss(red_model, device, train_loader))
test_loss.append(compute_loss(red_model, device, test_loader))

        
epochs = 10
filename = './cifar10_VGG16_RedNet'+\
            '_cutIDx_%d.pth'%(cutoff_idx)

if os.path.isfile(filename):
    [rednet_pretrained, train_loss,test_loss] = torch.load(filename)
    red_model.load_state_dict(rednet_pretrained)
    print('rednet trained {} epoches is loaded'.format(epochs))
else:
    train_loss = []
    test_loss = []
    train_loss.append(compute_loss(red_model, device, train_loader))
    test_loss.append(compute_loss(red_model, device, test_loader))
    for epoch in range(1, epochs + 1):
        print('EPOCH {}'.format(epoch))
        train_loss.append(
                train_kd(red_model,
                model,
                device,
                train_loader,
                optimizer,
                train_max_batch=200,
                alpha=0.1,
                temperature=1.,
                epoch=epoch))
        test_loss.append(compute_loss(red_model, device, test_loader))
    torch.save([red_model.state_dict(), train_loss, test_loss], filename)


Accuracy of network on test images is 28.1250....count: 1
Accuracy of network on test images is 25.0000....count: 2
Accuracy of network on test images is 23.9583....count: 3
Accuracy of network on test images is 24.6094....count: 4
Accuracy of network on test images is 26.4062....count: 5
Accuracy of network on test images is 25.9797....count: 6
Pre nnz = 6.62, proj_model nnz=0.78, FNN nnz=0.0058
flops:  Pre = 190.51, proj_model = 0.20, FNN =0.00
Test Loss -0.0004433982982572938
 Top 1:  Accuracy: 674.0/2759 (24.43%)
Test Loss: -1.2233359048918737
Test Loss -0.0014386674701460749
 Top 1:  Accuracy: 179.0/689 (25.98%)
Test Loss: -0.9912418869306456
Test Loss -0.00044339831529596
 Top 1:  Accuracy: 674.0/2759 (24.43%)
Test Loss: -1.2233359519015536
Test Loss -0.0014386675846542515
 Top 1:  Accuracy: 179.0/689 (25.98%)
Test Loss: -0.9912419658267793
EPOCH 1


/scratch/lmeneghe/anaconda/envs/prova/lib/python3.8/site-packages/torch/nn/functional.py:2694: UserWarning: reduction: 'mean' divides the total loss by both the batch size and the support size.'batchmean' divides only by the batch size, and aligns with the KL div math definition.'mean' will be changed to behave the same as 'batchmean' in the next major release.
  warnings.warn(


Train Loss kd: 0.000284754115198349
Test Loss -0.002672288049586996
 Top 1:  Accuracy: 468.0/689 (67.92%)
Test Loss: -1.8412064661654401
EPOCH 2
Train Loss kd: 0.000178162539165147
Test Loss -0.003921203851779684
 Top 1:  Accuracy: 527.0/689 (76.49%)
Test Loss: -2.701709453876202
EPOCH 3
Train Loss kd: 0.00015652838631270805
Test Loss -0.004585435328241236
 Top 1:  Accuracy: 559.0/689 (81.13%)
Test Loss: -3.1593649411582114
EPOCH 4
Train Loss kd: 0.00013533468996540884
Test Loss -0.005252942879613351
 Top 1:  Accuracy: 577.0/689 (83.74%)
Test Loss: -3.6192776440535988
EPOCH 5
Train Loss kd: 0.00015816573173769058
Test Loss -0.005658894470373372
 Top 1:  Accuracy: 595.0/689 (86.36%)
Test Loss: -3.898978290087253
EPOCH 6
Train Loss kd: 0.00010584022049110582
Test Loss -0.006065483128377559
 Top 1:  Accuracy: 608.0/689 (88.24%)
Test Loss: -4.179117875452138
EPOCH 7
Train Loss kd: 0.00010270386901874826
Test Loss -0.006420820208728109
 Top 1:  Accuracy: 615.0/689 (89.26%)
Test Loss: -4.423